<a href="https://colab.research.google.com/github/prabhudc/DQ/blob/master/Predicting_the_stock_market.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting Stock Market

This project is to predict the stock prices of the S&P 500 Index. The prediction model will be built on historic data from 1950 until 2015.


The dataset has the following fields which would be use for model building

* Date -- The date of the record.
* Open -- The opening price of the day (when trading starts).
* High -- The highest trade price during the day.
* Low -- The lowest trade price during the day.
* Close -- The closing price for the day (when trading is finished).
* Volume -- The number of shares traded.
* Adj Close -- The daily closing price, adjusted retroactively to include any corporate actions. Read

## Libaries

In [150]:
# The standard
import pandas as pd
import numpy as np
import csv
from datetime import datetime

# Plotting
import matplotlib.pyplot as plt

# Metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error


# Models
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression


In [212]:
# Import the data
data = pd.read_csv("/content/drive/My Drive/DataScience/files/sphist.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16590 entries, 0 to 16589
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       16590 non-null  object 
 1   Open       16590 non-null  float64
 2   High       16590 non-null  float64
 3   Low        16590 non-null  float64
 4   Close      16590 non-null  float64
 5   Volume     16590 non-null  float64
 6   Adj Close  16590 non-null  float64
dtypes: float64(6), object(1)
memory usage: 907.4+ KB


## Data Cleaning

In [213]:
# Change the datatype of the field "Date" from object to Date
data['Date'] = pd.to_datetime(data['Date']).copy()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16590 entries, 0 to 16589
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       16590 non-null  datetime64[ns]
 1   Open       16590 non-null  float64       
 2   High       16590 non-null  float64       
 3   Low        16590 non-null  float64       
 4   Close      16590 non-null  float64       
 5   Volume     16590 non-null  float64       
 6   Adj Close  16590 non-null  float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 907.4 KB


In [214]:
# Sort the dataset by datetime
data.sort_values(by=['Date'],inplace=True,ignore_index=True)


## Feature Selection

In [215]:
df = data[['Date','Close']].rolling(window=5,min_periods=5,on='Date', closed='right').mean()
# df.set_index( df['Close'].index.values + 1,inplace=True)

data['avg_price_5'] = df.shift(periods=1)['Close']
data.head(10)

,Date,Open,High,Low,Close,Volume,Adj Close,avg_price_5
0,1950-01-03,16.660000,16.660000,16.660000,16.660000,1260000.0,16.660000,NaN
1,1950-01-04,16.850000,16.850000,16.850000,16.850000,1890000.0,16.850000,NaN
2,1950-01-05,16.930000,16.930000,16.930000,16.930000,2550000.0,16.930000,NaN
3,1950-01-06,16.980000,16.980000,16.980000,16.980000,2010000.0,16.980000,NaN
4,1950-01-09,17.080000,17.080000,17.080000,17.080000,2520000.0,17.080000,NaN
5,1950-01-10,17.030001,17.030001,17.030001,17.030001,2160000.0,17.030001,16.900
6,1950-01-11,17.090000,17.090000,17.090000,17.090000,2630000.0,17.090000,16.974
7,1950-01-12,16.760000,16.760000,16.760000,16.760000,2970000.0,16.760000,17.022
8,1950-01-13,16.670000,16.670000,16.670000,16.670000,3330000.0,16.670000,16.988
9,1950-01-16,16.719999,16.719999,16.719999,16.719999,1460000.0,16.719999,16.926


In [216]:
def create_feature(field,func, window):
  # Create rolling window
  df = data[['Date',field]].rolling(window=window,min_periods=window,on='Date', closed='right')

  # Apply summarization
  if func == 'mean':
    df = df.mean().copy()
  elif func == 'std':
    df = df.std().copy()  

  # Rest the index forward by 1
  # We want to predict the next day of the window
  df = df.shift(periods=1).copy()

  # Summarization values
  summary =  df[field].to_list()

  return  summary

In [226]:
# Mean, Closing Price
data['avg_price_5'] = create_feature('Close','mean',5)
data['avg_price_10'] = create_feature('Close','mean' ,10)
data['avg_price_30'] = create_feature('Close','mean',30)
data['avg_price_365'] = create_feature('Close','mean',365)
# Mean, Volume
data['avg_vol_5'] = create_feature('Volume','mean',5)
data['avg_vol_10'] = create_feature('Volume','mean' ,10)
data['avg_vol_30'] = create_feature('Volume','mean',30)
data['avg_vol_365'] = create_feature('Volume','mean',365)

# Standard Deviation, Closing Prices
data['std_price_5'] = create_feature('Close','std',5)
data['std_price_10'] = create_feature('Close','std' ,10)
data['std_price_30'] = create_feature('Close','std',30)
data['std_price_365'] = create_feature('Close','std',365)
# Standard Deviation, Volume
data['std_vol_5'] = create_feature('Volume','std',5)
data['std_vol_10'] = create_feature('Volume','std' ,10)
data['std_vol_30'] = create_feature('Volume','std',30)
data['std_vol_365'] = create_feature('Volume','std',365)

# Ratio between mean volume last 5 days and mean volume last year
data['ratio_avg_vol_5_365'] = data['avg_vol_5']/data['avg_vol_365']

# Ratio between std volume last 5 days and std volume last year
data['ratio_std_vol_5_365'] = data['std_vol_5']/data['std_vol_365']

# Component of date
data['year'] = data['Date'].dt.year
data['day_of_week'] = data['Date'].dt.weekday
data['month'] = data['Date'].dt.month



# Print data
data.head(10)

,Date,Open,High,Low,Close,Volume,Adj Close,avg_price_5,avg_price_10,avg_price_30,avg_price_365,avg_vol_5,avg_vol_10,avg_vol_30,avg_vol_365,std_price_5,std_price_10,std_price_30,std_price_365,std_vol_5,std_vol_10,std_vol_30,std_vol_365,ratio_avg_vol_5_365,ratio_std_vol_5_365,year,day_of_week,month
365,1951-06-19,22.020000,22.020000,22.020000,22.020000,1100000.0,22.020000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1951,1,6
366,1951-06-20,21.910000,21.910000,21.910000,21.910000,1120000.0,21.910000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1951,2,6
367,1951-06-21,21.780001,21.780001,21.780001,21.780001,1100000.0,21.780001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1951,3,6
368,1951-06-22,21.549999,21.549999,21.549999,21.549999,1340000.0,21.549999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1951,4,6
369,1951-06-25,21.290001,21.290001,21.290001,21.290001,2440000.0,21.290001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1951,0,6
370,1951-06-26,21.299999,21.299999,21.299999,21.299999,1260000.0,21.299999,21.710,NaN,NaN,NaN,1420000.0,NaN,NaN,NaN,0.292831,NaN,NaN,NaN,579137.289423,NaN,NaN,NaN,NaN,NaN,1951,1,6
371,1951-06-27,21.370001,21.370001,21.370001,21.370001,1360000.0,21.370001,21.566,NaN,NaN,NaN,1452000.0,NaN,NaN,NaN,0.278980,NaN,NaN,NaN,561177.333826,NaN,NaN,NaN,NaN,NaN,1951,2,6
372,1951-06-28,21.100000,21.100000,21.100000,21.100000,1940000.0,21.100000,21.458,NaN,NaN,NaN,1500000.0,NaN,NaN,NaN,0.208015,NaN,NaN,NaN,535350.352573,NaN,NaN,NaN,NaN,NaN,1951,3,6
373,1951-06-29,20.959999,20.959999,20.959999,20.959999,1730000.0,20.959999,21.322,NaN,NaN,NaN,1668000.0,NaN,NaN,NaN,0.162080,NaN,NaN,NaN,509627.314810,NaN,NaN,NaN,NaN,NaN,1951,4,6
374,1951-07-02,21.100000,21.100000,21.100000,21.100000,1350000.0,21.100000,21.204,NaN,NaN,NaN,1746000.0,NaN,NaN,NaN,0.169205,NaN,NaN,NaN,475583.851702,NaN,NaN,NaN,NaN,NaN,1951,0,7


Drop the null values

In [227]:
data.dropna(inplace=True)
data

,Date,Open,High,Low,Close,Volume,Adj Close,avg_price_5,avg_price_10,avg_price_30,avg_price_365,avg_vol_5,avg_vol_10,avg_vol_30,avg_vol_365,std_price_5,std_price_10,std_price_30,std_price_365,std_vol_5,std_vol_10,std_vol_30,std_vol_365,ratio_avg_vol_5_365,ratio_std_vol_5_365,year,day_of_week,month
730,1952-12-05,25.620001,25.620001,25.620001,25.620001,1.510000e+06,25.620001,25.680000,25.525000,24.835000,23.820712,1.810000e+06,1.850000e+06,1.584000e+06,1.356904e+06,0.049497,0.180508,0.612585,0.982384,2.933428e+05,2.226607e+05,4.015539e+05,3.456496e+05,1.333919,0.848671,1952,4,12
731,1952-12-08,25.760000,25.760000,25.760000,25.760000,1.790000e+06,25.760000,25.672000,25.559000,24.886667,23.830575,1.680000e+06,1.827000e+06,1.601333e+06,1.358027e+06,0.056302,0.160100,0.611202,0.982327,2.383275e+05,2.459471e+05,3.859492e+05,3.454786e+05,1.237088,0.689847,1952,0,12
732,1952-12-09,25.930000,25.930000,25.930000,25.930000,2.120000e+06,25.930000,25.688000,25.608000,24.952000,23.841123,1.618000e+06,1.830000e+06,1.622333e+06,1.359863e+06,0.069065,0.134808,0.595595,0.982318,1.044988e+05,2.452210e+05,3.781688e+05,3.459900e+05,1.189826,0.302028,1952,1,12
733,1952-12-10,25.980000,25.980000,25.980000,25.980000,1.880000e+06,25.980000,25.726000,25.659000,25.020667,23.852493,1.720000e+06,1.832000e+06,1.651000e+06,1.362658e+06,0.130115,0.151250,0.585208,0.982413,2.467793e+05,2.477364e+05,3.823282e+05,3.479987e+05,1.262239,0.709138,1952,2,12
734,1952-12-11,25.959999,25.959999,25.959999,25.959999,1.790000e+06,25.959999,25.780000,25.721000,25.085667,23.864630,1.774000e+06,1.827000e+06,1.678333e+06,1.364137e+06,0.171318,0.141849,0.579647,0.981253,2.462316e+05,2.460375e+05,3.676494e+05,3.490484e+05,1.300456,0.705437,1952,3,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16585,2015-12-01,2082.929932,2103.370117,2082.929932,2102.629883,3.712120e+09,2102.629883,2087.024023,2079.273999,2073.984998,2035.531178,3.207544e+09,3.569041e+09,3.842181e+09,3.527800e+09,3.916109,14.883905,24.654181,64.370261,1.099518e+09,8.509271e+08,5.985570e+08,6.938227e+08,0.909219,1.584724,2015,1,12
16586,2015-12-02,2101.709961,2104.270020,2077.110107,2079.510010,3.950640e+09,2079.510010,2090.231982,2084.217993,2076.283993,2035.914082,3.232372e+09,3.566129e+09,3.856341e+09,3.526090e+09,7.956808,13.393465,23.970453,64.352527,1.111591e+09,8.503219e+08,5.899408e+08,6.925928e+08,0.916702,1.604970,2015,2,12
16587,2015-12-03,2080.709961,2085.000000,2042.349976,2049.620117,4.306490e+09,2049.620117,2088.306006,2087.125000,2077.908659,2036.234356,3.245514e+09,3.518458e+09,3.876979e+09,3.529468e+09,9.333599,6.759073,22.378095,64.277554,1.121578e+09,8.090357e+08,5.817195e+08,6.916451e+08,0.919548,1.621610,2015,3,12
16588,2015-12-04,2051.239990,2093.840088,2051.239990,2091.689941,4.214910e+09,2091.689941,2080.456006,2083.729004,2078.931331,2036.507343,3.536224e+09,3.556468e+09,3.899603e+09,3.532802e+09,19.599946,13.702747,20.183769,64.121622,1.181180e+09,8.387151e+08,5.848831e+08,6.924515e+08,1.000969,1.705795,2015,4,12


### Model Building

In [228]:
train = data[data['Date'] < datetime(year=2013,month=1,day=1)]
test =  data[data['Date'] >= datetime(year=2013,month=1,day=1)]

# Drop the unnecessary columns
train.drop(columns=['Date','Open','High','Low','Volume','Adj Close'],inplace=True)
test.drop(columns=['Date','Open','High','Low','Volume','Adj Close'],inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [229]:
print('Training dataset dimensions', train.shape)
print('Test dataset dimensions', test.shape)

Training dataset dimensions (15121, 22)
Test dataset dimensions (739, 22)


In [230]:
# Set-up X and y for model execution
train_X = train[[col for col in train.columns if col != 'Close']]
train_y = train['Close']

test_X = test[[col for col in test.columns if col != 'Close']]
test_y = test['Close']

In [231]:
linear_model = LinearRegression()
linear_model.fit(X=train_X, y=train_y)
y_pred = linear_model.predict(test_X)
mse = mean_squared_error(y_true=test_y, y_pred=y_pred)
mae = mean_absolute_error(y_true = test_y, y_pred=y_pred)
print("Model RMSE = ", np.sqrt(mse))
print("Model MAE = ", mae)

# Run 1 with only CLosing Prices  : 21, 15
# Run 2 with including Volume : 21, 15
# Run 3 including ratio avg vol of last 5 over 365 : 21, 15

Model RMSE =  21.964538507418034
Model MAE =  15.676871106099968
